# **P**rincipal **O**rthogonal **L**atent **C**omponents **A**nalysis Net (POLCA-Net)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn
plt.style.use("seaborn-v0_8-paper")
import numpy as np

import torch
from sklearn import datasets, decomposition

In [3]:
from polcanet import PolcaNet, LinearDecoder
from polcanet.example_aencoders import autoencoder_factory, StandardScalerTorch, MinMaxScalerTorch

In [4]:
from polcanet.polcanet_reports import analyze_latent_space, show_correlation_matrix, plot_scatter_corr_matrix, plot_stdev_pct, \
    plot_cumsum_variance, analyze_latent_feature_importance, analyze_reconstruction_error, orthogonality_test_analysis, \
    variance_test_analysis, linearity_tests_analysis

In [5]:
np.random.seed(5)

In [6]:
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

### Load dataset

In [7]:
digits = datasets.load_digits()
X = digits.data / 255
y = digits.target
X.shape,X[0].shape

((1797, 64), (64,))

### Fit standard sklearn PCA

In [8]:
pca = decomposition.PCA(n_components=8)
pca.fit(X)
Xpca = pca.transform(X)
pca.explained_variance_ratio_

array([0.14890594, 0.13618771, 0.11794594, 0.08409979, 0.05782413,
       0.04916901, 0.04315983, 0.03661299])

### Fit POLCANet

In [ ]:
ae_input = X
act_fn = torch.nn.Mish()
input_dim = (ae_input.shape[1],)
latent_dim = 64  # Hey! ... let the kids alone!

encoder = autoencoder_factory(
    input_dim=input_dim,
    latent_dim=latent_dim,
    hidden_dim=512,
    num_layers=20,
    autoencoder_type="dense",
    act_fn=act_fn,
)

decoder = LinearDecoder(latent_dim=latent_dim,
                             input_dim=input_dim, 
                             hidden_dim=512, 
                             num_layers=5)


model = PolcaNet(
    encoder=encoder,
    decoder=decoder,
    latent_dim = latent_dim,
    alpha=0.1,  # ortgogonality loss
    beta=1.0,  # variance sorting loss
    gamma=1.0,  # variance reduction loss
    device="cuda",
    scaler = MinMaxScalerTorch(),
)
model

In [ ]:
model.to("cuda")
model.train_model(data=X, batch_size= 512, num_epochs=10000, report_freq=100, lr=1e-3)

In [ ]:
model.train_model(data=X, batch_size= 512, num_epochs=10000, report_freq=100, lr=1e-4)

In [ ]:
model.train_model(data=X, batch_size= 512, num_epochs=10000, report_freq=100, lr=1e-5)

## Evaluate results

In [ ]:
analyze_reconstruction_error(model, X)

In [ ]:
latents, reconstructed = model.predict(X)

In [ ]:
analyze_latent_space(model, latents=latents)

In [ ]:
orthogonality_test_analysis(model, X)

In [ ]:
variance_test_analysis(model, X)

In [ ]:
linearity_tests_analysis(model, X)

In [ ]:
plot_cumsum_variance(model,X)

## Polca Net vs. PCA

In [ ]:
def plot2d_analysis(X, y, title, legend=True):
    fig = plt.figure(1, figsize=(5, 5))
    ax = fig.add_subplot(111)

    for label in range(10):
        ax.scatter(X[y == label, 0], X[y == label, 1], label=label)
        ax.set_xlabel("component: 0")
        ax.set_ylabel("component 1")        
    if legend:
        plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
    plt.title(title)
    plt.show()
    return fig, ax

In [ ]:
o1 = widgets.Output()
o2 = widgets.Output()
with o1:
    _,_ = plot2d_analysis(Xpca, y, title="PCA transform", legend=True)
with o2:
    _,_ = plot2d_analysis(latents, y, title="POLCA-Net latent")
layout = widgets.Layout(grid_template_columns="repeat(2, 600px)")
accordion = widgets.GridBox(children=[o1, o2], layout=layout)
display(accordion)

In [ ]:
o1 = widgets.Output()
o2 = widgets.Output()
o3 = widgets.Output()
o4 = widgets.Output()

with o1:
    fig1, ax1 = plot2d_analysis(X, y, "Original data two first componets", legend=False)

with o2:
    latents, reconstructed = model.predict(X)
    fig2, ax2 = plot2d_analysis(np.round(reconstructed, 1), y, title="Reconstructed with POLCA all componets", legend=False)

with o3:
    latents, reconstructed = model.predict(X)
    fig3, ax3 = plot2d_analysis(np.round(reconstructed, 1), y, title="Reconstructed with POLCA two componets", legend=False)

with o4:
    fig4, ax4 = plot2d_analysis(np.round(pca.inverse_transform(Xpca),1), y, "Reconstructed with PCA two componets", legend=False)
    


layout = widgets.Layout(grid_template_columns="repeat(2, 450px)")
accordion = widgets.GridBox(children=[o1, o2, o3, o4], layout=layout)
display(accordion)

In [ ]:
latents, reconstructed = model.predict(X)
vectors = []
labels = ["Setosa", "Versicolour", "Virginica"]
for c, label in enumerate(labels):
    vectors.append(np.sum(latents[y == c, :], axis=1))


plt.boxplot(vectors, labels=labels)
plt.violinplot(vectors, showmeans=False, showmedians=True)
plt.suptitle("Polca Analysis of the summation of latent orthogonal components")
plt.show()

In [ ]:
import seaborn as sns

o1 = widgets.Output()
o2 = widgets.Output()


with o1:
    scores = model.score(X)
    sns.displot(scores, kde=True)
    plt.title("Last component with clean data")
    plt.show()

with o2:
    scores = model.score(X * (np.random.random(size=X.shape) - 0.5) * 1)
    sns.displot(scores, kde=True)
    plt.title("Last componet with uniform noise in data")
    plt.show()


layout = widgets.Layout(grid_template_columns="repeat(2, 500px)")
accordion = widgets.GridBox(children=[o1, o2], layout=layout)
display(accordion)

In [ ]:
model.std_metrics

In [ ]:
model.mean_metrics

## Test Classification with two components on PCA vs POLCA Net

In [ ]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
from scipy.stats import ttest_rel

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
X_train_pca.shape, X_test_pca.shape

In [ ]:
# Transform the data using POLCA-Net
#X_train_polca = model.predict(X_train,np.array([1, 1, 0, 0]))[0][:,:2]
X_train_polca = model.predict(X_train)[0][:,:8]
#X_test_polca = model.predict(X_test, np.array([1, 1, 0, 0]))[0][:,:2]
X_test_polca = model.predict(X_test)[0][:,:8]
X_train_polca.shape, X_test_polca.shape

In [ ]:
# Define classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Gaussian Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Ridge Classifier': RidgeClassifier(),
    'Perceptron': Perceptron()
}

In [ ]:
# Train and evaluate classifiers on both PCA and POLCA-Net transformed datasets
results = []

for name, clf in classifiers.items():
    # Train on PCA
    clf.fit(X_train_pca, y_train)
    y_pred_pca = clf.predict(X_test_pca)
    accuracy_pca = accuracy_score(y_test, y_pred_pca)
    report_pca = classification_report(y_test, y_pred_pca, output_dict=True)
    cm_pca = confusion_matrix(y_test, y_pred_pca)
    
    # Train on POLCA-Net
    clf.fit(X_train_polca, y_train)
    y_pred_polca = clf.predict(X_test_polca)
    accuracy_polca = accuracy_score(y_test, y_pred_polca)
    report_polca = classification_report(y_test, y_pred_polca, output_dict=True)
    cm_polca = confusion_matrix(y_test, y_pred_polca)
    
    # Append results
    results.append({
        'Classifier': name,
        'Transformation': 'PCA',
        'Accuracy': accuracy_pca,
        'Precision': report_pca['weighted avg']['precision'],
        'Recall': report_pca['weighted avg']['recall'],
        'F1-Score': report_pca['weighted avg']['f1-score'],
        'Confusion Matrix': cm_pca
    })
    
    results.append({
        'Classifier': name,
        'Transformation': 'POLCA-Net',
        'Accuracy': accuracy_polca,
        'Precision': report_polca['weighted avg']['precision'],
        'Recall': report_polca['weighted avg']['recall'],
        'F1-Score': report_polca['weighted avg']['f1-score'],
        'Confusion Matrix': cm_polca
    })

In [ ]:
# Create a DataFrame to display the results
results_df = pd.DataFrame(results)

# Display the main metrics table
main_metrics_df = results_df.drop(columns=['Confusion Matrix'])
main_metrics_df

In [ ]:
# Statistical test: Paired t-test for accuracies
pca_accuracies = results_df[results_df['Transformation'] == 'PCA']['F1-Score']
polca_accuracies = results_df[results_df['Transformation'] == 'POLCA-Net']['F1-Score']

t_stat, p_value = ttest_rel(pca_accuracies.values, polca_accuracies.values)

print(f"\nPaired t-test results: t-statistic = {t_stat}, p-value = {p_value}")

if p_value < 0.05:
    print("There is a statistically significant difference between the PCA and POLCA-Net transformations.")
else:
    print("There is no statistically significant difference between the PCA and POLCA-Net transformations.")


In [ ]:
# Plotting the results
plt.figure(figsize=(10, 4))

# Plot PCA
plt.subplot(1, 2, 1)
plt.scatter(X_test_pca[:, 0], X_test_pca[:, 1], c=y_test, cmap='viridis', edgecolor='k', s=50)
plt.title('PCA: Iris Test Set')
plt.xlabel('Component 1')
plt.ylabel('Component 2')

# Plot POLCA-Net
plt.subplot(1, 2, 2)
plt.scatter(X_test_polca[:, 0], X_test_polca[:, 1], c=y_test, cmap='viridis', edgecolor='k', s=50)
plt.title('POLCA-Net: Iris Test Set')
plt.xlabel('Component 1')
plt.ylabel('Component 2')

plt.show()

# Plot Confusion Matrices for each classifier
fig, axes = plt.subplots(len(classifiers), 2, figsize=(10, 20))

for i, (name, clf) in enumerate(classifiers.items()):
    # PCA Confusion Matrix
    cm_pca = results_df[(results_df['Classifier'] == name) & (results_df['Transformation'] == 'PCA')]['Confusion Matrix'].values[0]
    axes[i, 0].imshow(cm_pca, interpolation='nearest', cmap=plt.cm.Blues)
    axes[i, 0].set_title(f'{name} Confusion Matrix - PCA')
    axes[i, 0].set_xlabel('Predicted label')
    axes[i, 0].set_ylabel('True label')
    
    # POLCA-Net Confusion Matrix
    cm_polca = results_df[(results_df['Classifier'] == name) & (results_df['Transformation'] == 'POLCA-Net')]['Confusion Matrix'].values[0]
    axes[i, 1].imshow(cm_polca, interpolation='nearest', cmap=plt.cm.Blues)
    axes[i, 1].set_title(f'{name} Confusion Matrix - POLCA-Net')
    axes[i, 1].set_xlabel('Predicted label')
    axes[i, 1].set_ylabel('True label')

plt.tight_layout()
plt.show()